# Moment models fine-tune analysis | Mask test
> This notebook is the pre-analysis of moment models to select the cases used in the paper.

# RECORDATORIO 
1) Conseguir que compilen de nuevo
2) Enmascarar también en evaluación, no solo en training, majaretilla...

In [1]:
# Input parameters
model_patch_size = 8
verbose          = 0
reset_kernel     = False

In [2]:
#! pip install --no-deps ydata_profiling
#! pip install --no-deps dacite
#! pip install --no-deps multimethod
#! pip install --no-deps visions
#! pip install --no-deps wordcloud
#! pip install --no-deps imagehash
#! pip install --no-deps htmlmin

In [3]:
# Imports
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut
from dvats.imports import beep
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
import ydata_profiling as ydp
import seaborn as sns

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<froze


Octave is ready <oct2py.core.Oct2Py object at 0x7f1848bc9630>

















In [4]:
import torch
torch.cuda.set_device(0)
#torch.cuda.set_device(1)

In [5]:
from fastai.losses import MSELossFlat
from dvats.encoder import MAELossFlat, EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE
import dvats.config as cfg_
import gc
import os

### Setting up Weight & Biases information

In [6]:
# User and project
entity = os.environ.get("WANDB_ENTITY")
project = os.environ.get("WANDB_PROJECT")
folder = entity+'/'+project+'/'

# Dataset
dataset = 'gtrends_kohls'
dataset_version = 'v2'
enc_artifact_dataset = folder + dataset + ':' + dataset_version

# Models
model_family = 'zeroshot-moment'
task = 'embedding'
enc_artifact_small_name = folder + model_family + '-small-' + task + ':v0'
enc_artifact_base_name  =  folder + model_family + '-base-' + task + ':v0'
enc_artifact_large_name = folder + model_family + '-large-' + task + ':v0'

### Download dataset

In [7]:
print("Getting dataset artifact: ", enc_artifact_dataset)
df_artifact = wandb_api.artifact(enc_artifact_dataset, type = 'dataset')

Getting dataset artifact:  mi-santamaria/deepvats/gtrends_kohls:v2


In [8]:
print(df_artifact.name)
df = df_artifact.to_df()
display(df.head())
df.shape

gtrends_kohls:v2


wandb:   1 of 1 files downloaded.  


,volume
2004-01-01,0.010417
2004-01-08,0.010417
2004-01-15,0.010417
2004-01-22,0.000000
2004-01-29,0.000000


(440, 1)

### Encoder Input

In [9]:
enc_input, _ = SlidingWindow(window_len=17, stride=2, get_y=[])(df)
enc_input.shape

(212, 1, 17)

### Functions

In [10]:
def count_parameters(model):
    #return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for p in model.parameters())

def greater_than(lst, val):
    vals = []
    for x in lst:
        try:
            x = int(x)
            if (x > val): 
                vals.append(x)
        except:
            continue
    return vals
    #return [ x for x in lst if isinstance(x, int) and x > val]

In [11]:
def plot_with_multiple_secondary_y(df, primary_vars, secondary_vars, figsize=(12, 6)):
    """
    Plots multiple variables with different scales on primary and secondary y-axes.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - primary_vars (list): Variables to plot on the primary y-axis.
    - secondary_vars (list): Variables to plot on the secondary y-axis.
    - figsize (tuple): Size of the figure.

    Returns:
    - None: Displays the plot.
    """
    ax = df[primary_vars + secondary_vars].plot(
        secondary_y=secondary_vars, figsize=figsize
    )
    ax.set_title("Variables with Primary and Secondary Axes")
    plt.show()

In [12]:
def plot_correlation(profile, figsize=(8, 6)):
    correlation_matrix = profile.corr()
    # Crear el heatmap con seaborn
    plt.figure(figsize = figsize)  # Ajusta el tamaño si es necesario
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
    plt.title("Correlation Matrix")
    plt.show()

### Common Fine-tune args

In [13]:
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)

In [14]:
print(config['batch_size'])
print(config['r'])
print(config['analysis_mode'])

16
0.4
online


In [15]:
common_args = {
    "X": df,
    "stride": 1,
    "batch_size": config['batch_size'],
    "cpu": False,
    "to_numpy": False,
    "time_flag": True,
    "n_windows": None,
    "n_windows_percent": None,
    "shot": True,
    "eval_pre": True,
    "eval_post": True,
    "lr": config['r'], #use enc_run lr,
    "lr_scheduler_flag": False,
    "lr_scheduler_name": "cosine_with_restarts",
    "lr_scheduler_num_warmup_steps": None,
    "window_sizes": [12],
    "full_dataset": True,
    "window_sizes_offset": 0.05,
    "windows_min_distance": 0, #int(np.ceil(1.5*enc_input.shape[0]/100)),
    "print_to_path": True,
    "print_both": True,
    "print_path": "./logs.txt",
    "print_mode": "w",
    "use_moment_masks": True,
    "mask_stateful": config['mask_stateful'],
    "mask_future": config['mask_future'],
    "mask_sync": config['mask_sync'],
    "analysis_mode": config['analysis_mode'],
    "use_wandb": config['use_wandb'],
    "norm_by_sample": config['norm_by_sample'],
    "norm_use_single_batch": config['norm_use_single_batch'],
    "show_plot": True,
    "metrics": [EvalMSE, EvalRMSE, EvalMAE, EvalSMAPE],
    "metrics_args": [{'squared': False}, {'squared': True}, {}, {}],
    "metrics_names":["mse", "rmse", "mae", "smape"],
    "metrics_dict": None,
    "criterion": torch.nn.MSELoss()
}

In [16]:
common_args["windows_min_distance"]

0

### Cases execution function

In [17]:
from momentfm.utils.masking import Masking 

In [18]:
def cases_loop(
    model, 
    n_epochs_list, 
    dataset_percents, 
    maskared_percents, 
    n_sizes_list, 
    summarized = True, 
    do_beep = True, 
    verbose = 1,
    save = True,
    file_errors = "",
    file_results = ""
):
    mssg = ut.Mssg(verbose = verbose, level = -1)
    result_columns = [
        'model_size','n_epochs','dataset_percent','maskared_percent','n_windows', 
        'time',
        'first_train_loss','first_mse','first_rmse','first_mae','first_smape', 
        'last_train_loss','last_mse','last_rmse','last_mae','last_smape'
    ]
    result_columns = result_columns if summarized else result_columns + ['losses','eval_results_pre','eval_results_post']
    results = pd.DataFrame(columns = result_columns)
    
    errors = pd.DataFrame(
        columns = [
            'model_size',
            'n_epochs',
            'dataset_percent',
            'maskared_percent',
            'n_windows',
            'windows',
            'error'
        ]
    )
    model_backup = deepcopy(model)
    i = 0
    for n_epochs in n_epochs_list:
        for dataset_percent in dataset_percents:
            print(dataset_percent)
            for maskared_percent in maskared_percents:
                model.mask_generator = Masking(mask_ratio = maskared_percent)
                for sizes in n_sizes_list:
                    print(f"--> epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}")
                    print(f" sizes {sizes}")
                    print(f"Cuda memmory allocated: {torch.cuda.memory_allocated()}")
                    model_case = deepcopy(model_backup)
                    case = {
                            'model_size': "small",
                            'n_epochs': n_epochs,
                            'dataset_percent': dataset_percent,
                            'maskared_percent': maskared_percent,
                            'n_windows': sizes,
                            'windows': None
                           }
                    result_dict = deepcopy(case)
                    error_dict = deepcopy(case)
                    error = False
                    print(1-dataset_percent)
                    torch.cuda.synchronize()
                    result = fine_tune(
                        enc_learn           = model_case,
                        window_mask_percent = maskared_percent,
                        training_percent    = dataset_percent,
                        validation_percent  = 0.3,
                        num_epochs          = n_epochs,
                        n_window_sizes      = sizes,
                        verbose             = verbose,
                        register_errors     = True,
                        save_best_or_last   = True, # only available for moment
                        **common_args    
                    )
                    try:
                        internal_errors = result[10]
                    except:
                        internal_errors = pd.DataFrame(columns=["window", "error"])
                    if len(result[0]) > 0:
                        result_dict.update({
                            'time'             : result[4],
                            'windows'          : result[8].cpu() if isinstance(result[8], torch.Tensor) else result[8],
                            'first_train_loss' : result[0][0][0].cpu().item() if torch.is_tensor(result[0][0][0]) else result[0][0][0],
                            'last_train_loss'  : result[0][-1][-1].cpu().item() if torch.is_tensor(result[0][-1][-1]) else result[0][-1][-1],
                            'first_eval_loss'  : result[1]['loss'].cpu().item() if torch.is_tensor(result[1]['loss']) else result[1]['loss'],
                            'last_eval_loss'   : result[2]['loss'][-1].cpu().item() if torch.is_tensor(result[2]['loss'][-1]) else result[2]['loss'][-1],
                            'first_mse'        : result[1]['mse'].cpu().item() if torch.is_tensor(result[1]['mse']) else result[1]['mse'],
                            'last_mse'         : result[2]['mse'][-1].cpu().item() if torch.is_tensor(result[2]['mse'][-1]) else result[2]['mse'][-1],
                            'first_rmse'       : result[1]['rmse'].cpu().item() if torch.is_tensor(result[1]['rmse']) else result[1]['rmse'],
                            'last_rmse'        : result[2]['rmse'][-1].cpu().item() if torch.is_tensor(result[2]['rmse'][-1]) else result[2]['rmse'][-1],
                            'first_mae'        : result[1]['mae'].cpu().item() if torch.is_tensor(result[1]['mae']) else result[1]['mae'],
                            'last_mae'         : result[2]['mae'][-1].cpu().item() if torch.is_tensor(result[2]['mae'][-1]) else result[2]['mae'][-1],
                            'first_smape'      : result[1]['smape'].cpu().item() if torch.is_tensor(result[1]['smape']) else result[1]['smape'],
                            'last_smape'       : result[2]['smape'][-1].cpu().item() if torch.is_tensor(result[2]['smape'][-1]) else result[2]['smape'][-1],
                            'best_epochs'       : result[9],
                            'train_losses'      : result[0][0],
                            'eval_pre'          : result[1],
                            'eval_post'         : result[2],
                            'full_result'       : result
                        })
        
                        if not summarized:
                            result_dict.update({
                                'losses'           : [[v.cpu().item() if torch.is_tensor(v) else v for v in loss] for loss in result[0]],
                                'eval_results_pre' : {k: v.cpu().item() if torch.is_tensor(v) else v for k, v in result[1].items()},
                                'eval_results_post': {k: v.cpu().item() if torch.is_tensor(v) else v for k, v in result[2].items()},
                                })  
                        results = pd.concat([results, pd.DataFrame([result_dict])], ignore_index=True)
                    else:
                        print("Failed case")
                        # Attach possible errors
                        internal_errors['model_size'] = case['model_size']
                        internal_errors['n_epochs'] = case['n_epochs']
                        internal_errors['dataset_percent'] = case['dataset_percent']
                        internal_errors['maskared_percent'] = case['maskared_percent']
                        internal_errors['windows'] =  result[8]
                        internal_errors['windows'] = [result[8]]*len(internal_errors)
                        errors = pd.concat([errors, internal_errors])
                    if not error: mssg.print_error(f" case {case} | time: {result[4]}")
                    before = torch.cuda.memory_allocated()
                    model_case = None
                    gc.collect()
                    torch.cuda.empty_cache()
                    display(results)
                    if do_beep:
                        beep(1)
                    mssg.print(f"epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent}, sizes {sizes} -->")
                if save:
                    results.to_csv(file_results, index = False, header = True)
                    errors.to_csv(file_errors, index = False, header = True)
                if do_beep:
                    beep(2)
                    beep(2)
                mssg.print(f"epoch {n_epochs}, dataset_percent {dataset_percent}, mask {maskared_percent} -->")
            if do_beep:
                beep(3)
                beep(3)
                beep(3)
            mssg.print(f"epoch {n_epochs}, dataset_percent {dataset_percent}-->")
        if do_beep:
            beep(4)
            beep(4)
            beep(4)
            beep(4)
        mssg.print(f"epoch {n_epochs}-->")
    if do_beep:
        beep(1000)
        beep(1000)
        beep(1000)
        beep(1000)
        beep(1000)
    model_backup = None
    gc.collect()
    torch.cuda.empty_cache()
    return results, errors

# Analysis

## Defining full reasonable values

The following parameters are modified within the fine-tuning:
- `n_epochs_list` is used to set up the number of epochs used in the training step.
- `dataset_percents` is used to select the percentage of the dataset used for each case fine-tuning.
- `maskared_percents` is used to select the  percentage of the training dataset we want to mask for the model to fill it up.
- `sizes` is used to select the number of window sizes we want to use for the fine-tuning.

In [19]:
n_epochs_list     = [5, 10, 20]
dataset_percents  = [0.25, 0.5, 0.75, 1] #1 No tendría sentido porque sería como hacer lo mismo que con mvp.. entrenar con todo el dataset.
maskared_percents = [0.25, 0.5, 0.75]
sizes             = [1, 5]
total_cases_small = len(n_epochs_list)*len(dataset_percents)*len(maskared_percents)*len(sizes)
print(f"Total cases: {total_cases_small}")

Total cases: 72


### Moment-Small
Getting the results for moment small.

#### Download the model

In [20]:
already_computed_small = False
#file_errors_small  = 'errors_small_29012025_1.csv'
#file_results_small = 'results_small_29012025_1.csv'
file_errors_small  = 'errors_small_01022025_1.csv'
file_results_small = 'results_small_01022025_1.csv'

In [21]:
print("Getting small artifact: ", enc_artifact_small_name)
enc_artifact_small = wandb_api.artifact(enc_artifact_small_name, type='learner')
print(enc_artifact_small.name)
moment_small = enc_artifact_small.to_obj()

Getting small artifact:  mi-santamaria/deepvats/zeroshot-moment-small-embedding:v0
zeroshot-moment-small-embedding:v0


wandb: Downloading large artifact zeroshot-moment-small-embedding:v0, 144.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


In [22]:
moment_small.head.linear.out_features

8

### Specific parameters

In [23]:
n_epochs_list_small    = [1, 5, 10] # [1, 5, 10, 20, 40, 80, 100]
dataset_percents_small = [0.25, 0.5, 0.75, 1] #1 No tendría sentido porque sería como hacer lo mismo que con mvp... entrenar con todo el dataset.
maskared_percents_small= [0.25, 0.5, 0.75] #[0, 0.25, 0.5, 0.75, 1]  #1 debería devolver fatal el loss
sizes_small            = [1, 5]
total_cases_small      = len(n_epochs_list_small)*len(dataset_percents_small)*len(maskared_percents_small)*len(sizes_small)
print(f"Total cases: {total_cases_small}")

Total cases: 72


### Execute!

In [ ]:
results_small      = None
errors_small       = None
if already_computed_small:
    results_small = pd.read_csv(file_results_small, index_col=None, header=0)
    errors_small  = pd.read_csv(file_errors_small, index_col=None, header=0)
else:
    results_small, errors_small = cases_loop(
        model             = moment_small, 
        n_epochs_list     = n_epochs_list_small, 
        dataset_percents  = dataset_percents_small, 
        maskared_percents = maskared_percents_small, 
        n_sizes_list      = sizes_small, 
        verbose           = 0,
        summarized        = True,
        save              = True,
        file_errors       = file_errors_small,
        file_results      = file_results_small
    )
    #already_computed_small = True

0.25
--> epoch 1, dataset_percent 0.25, mask 0.25
 sizes 1
Cuda memmory allocated: 0
0.75


100% 9/9 [00:05<00:00,  1.60it/s]
100% 5/5 [00:01<00:00,  3.09it/s]
100% 9/9 [00:04<00:00,  2.13it/s]

[0]  [ cases_loop ]  case {'model_size': 'small', 'n_epochs': 1, 'dataset_percent': 0.25, 'maskared_percent': 0.25, 'n_windows': 1, 'windows': None} | time: 1.693509817123413


,model_size,n_epochs,dataset_percent,maskared_percent,n_windows,time,first_train_loss,first_mse,first_rmse,first_mae,...,last_mae,last_smape,windows,first_eval_loss,last_eval_loss,best_epochs,train_losses,eval_pre,eval_post,full_result
0,small,1,0.25,0.25,1,1.69351,0.278592,0.050057,0.002575,0.029738,...,0.065003,0.527195,[12],0.002403,0.011882,[0],[0.2785921407630667],"{'mse': 0.050057176737342224, 'rmse': 0.0025754020427467906, 'mae': 0.029737689518802908, 'smape': 0.21658901563437807, 'loss': 0.002403268920412908}","{'mse': [0.105556974828863], 'rmse': [0.01299032673991015], 'mae': [0.06500311488513077], 'smape': [0.527194837719801], 'loss': [0.011881997964034477]}","([[0.2785921407630667]], {'mse': 0.050057176737342224, 'rmse': 0.0025754020427467906, 'mae': 0.029737689518802908, 'smape': 0.21658901563437807, 'loss': 0.002403268920412908}, {'mse': [0.105556974828863], 'rmse': [0.01299032673991015], 'mae': [0.06500311488513077], 'smape': [0.527194837719801], 'loss': [0.011881997964034477]}, [1.693509817123413], 1.693509817123413, [5.65706205368042, 4.2647435665130615], 9.921805620193481, MOMENTPipeline(\n (normalizer): RevIN()\n (tokenizer): Patching()\n (patch_embedding): PatchEmbedding(\n (value_embedding): Linear(in_features=8, out_features=512..."


[0]  [ cases_loop ] epoch 1, dataset_percent 0.25, mask 0.25, sizes 1 -->
--> epoch 1, dataset_percent 0.25, mask 0.25
 sizes 5
Cuda memmory allocated: 168909824
0.75


...

0.18744061460931793
0.043717231565261534
0.11833795497636856
0.6983802683322209
0.04371723247459158
[0]  [ cases_loop ]  case {'model_size': 'small', 'n_epochs': 1, 'dataset_percent': 1, 'maskared_percent': 0.25, 'n_windows': 5, 'windows': None} | time: 27.683936834335327


,model_size,n_epochs,dataset_percent,maskared_percent,n_windows,time,first_train_loss,first_mse,first_rmse,first_mae,...,last_mae,last_smape,windows,first_eval_loss,last_eval_loss,best_epochs,train_losses,eval_pre,eval_post,full_result
0,small,1,0.25,0.25,1,1.693510,0.278592,0.050057,0.002575,0.029738,...,0.065003,0.527195,[12],0.002403,0.011882,[0],[0.2785921407630667],"{'mse': 0.050057176737342224, 'rmse': 0.0025754020427467906, 'mae': 0.029737689518802908, 'smape': 0.21658901563437807, 'loss': 0.002403268920412908}","{'mse': [0.105556974828863], 'rmse': [0.01299032673991015], 'mae': [0.06500311488513077], 'smape': [0.527194837719801], 'loss': [0.011881997964034477]}","([[0.2785921407630667]], {'mse': 0.050057176737342224, 'rmse': 0.0025754020427467906, 'mae': 0.029737689518802908, 'smape': 0.21658901563437807, 'loss': 0.002403268920412908}, {'mse': [0.105556974828863], 'rmse': [0.01299032673991015], 'mae': [0.06500311488513077], 'smape': [0.527194837719801], 'loss': [0.011881997964034477]}, [1.693509817123413], 1.693509817123413, [5.65706205368042, 4.2647435665130615], 9.921805620193481, MOMENTPipeline(\n (normalizer): RevIN()\n (tokenizer): Patching()\n (patch_embedding): PatchEmbedding(\n (value_embedding): Linear(in_features=8, out_features=512..."
1,small,1,0.25,0.25,5,7.658274,0.104053,0.058667,0.003555,0.030022,...,0.197907,0.757459,"[8, 10, 12, 16, 17]",0.003284,0.198489,"[0, 0, 0, 0, 0]",[0.10405270835035481],"{'mse': 0.05866684906817518, 'rmse': 0.003554834314440888, 'mae': 0.03002238260098518, 'smape': 0.19264234298604702, 'loss': 0.0032838742352194255}","{'mse': [0.29885705643203153, 0.20866385111740815, 0.19162909040672282, 0.15014132540036856, 0.3213458578220087], 'rmse': [0.12347865735016345, 0.04746267129586409, 0.042432848823567654, 0.02872852415770987, 0.19848855467861315], 'mae': [0.17812061785505368, 0.13247015888247493, 0.11558274264165828, 0.09213574235741362, 0.19790694841503864], 'smape': [0.8281538167187339, 0.7915033292132713, 0.7641304308435609, 0.597862523405075, 0.7574591917837966], 'loss': [0.11435014340612623, 0.043849107498923935, 0.04086191771138045, 0.02872852251311997, 0.19848855631425977]}","([[0.10405270835035481], [0.09756046012043953], [0.15938336905092002], [0.4687319681048393], [0.5205669485032558]], {'mse': 0.05866684906817518, 'rmse': 0.003554834314440888, 'mae': 0.03002238260098518, 'smape': 0.19264234298604702, 'loss': 0.0032838742352194255}, {'mse': [0.29885705643203153, 0.20866385111740815, 0.19162909040672282, 0.15014132540036856, 0.3213458578220087], 'rmse': [0.12347865735016345, 0.04746267129586409, 0.042432848823567654, 0.02872852415770987, 0.19848855467861315], 'mae': [0.17812061785505368, 0.13247015888247493, 0.11558274264165828, 0.09213574235741362, 0.1979069..."
2,small,1,0.25,0.50,1,1.532157,0.524085,0.058976,0.003615,0.030481,...,0.587378,1.783032,[12],0.003306,0.836448,[0],[0.5240851042792201],"{'mse': 0.058976207855801155, 'rmse': 0.0036148822071356357, 'mae': 0.030481205917461666, 'smape': 0.20638097793257798, 'loss': 0.003306265305986421}","{'mse': [0.9599394037425726], 'rmse': [0.9230014525468979], 'mae': [0.587377865042523], 'smape': [1.7830323247604154], 'loss': [0.8364482743458616]}","([[0.5240851042792201]], {'mse': 0.058976207855801155, 'rmse': 0.0036148822071356357, 'mae': 0.030481205917461666, 'smape': 0.20638097793257798, 'loss': 0.003306265305986421}, {'mse': [0.9599394037425726], 'rmse': [0.9230014525468979], 'mae': [0.587377865042523], 'smape': [1.7830323247604154], 'loss': [0.8364482743458616]}, [1.5321567058563232], 1.5321567058563232, [4.284701585769653, 4.2428672313690186], 8.527568817138672, MOMENTPipeline(\n (normalizer): RevIN()\n (tokenizer): Patching()\n (patch_embedding): PatchEmbedding(\n (value_embedding): Linear(in_features=8, out_features=512..."
3,small,1,0.25,0.50,5,7.538157,0.092932,0.048457,0.002439,0.024713,...,0.112508,0.743860,"[8, 10, 12, 16, 17]",0.002249,0.040398,"[0, 0, 0, 0, 0]",[0.0929318884620443],"{'mse': 0.04845714750109790

[0]  [ cases_loop ] epoch 1, dataset_percent 1, mask 0.25, sizes 5 -->


[0]  [ cases_loop ] epoch 1, dataset_percent 1, mask 0.25 -->
--> epoch 1, dataset_percent 1, mask 0.5
 sizes 1
Cuda memmory allocated: 3054448640
0


100% 9/9 [00:04<00:00,  2.70it/s]

Observaciones:
- El número de épocas más pequeño a partir de los 

In [ ]:
#results_small = pd.read_csv(file_results_small, index_col=None, header=0)
#errors_small  = pd.read_csv(file_errors_small, index_col=None, header=0)
print(errors_small.shape)
print(results_small.shape)

In [ ]:
print("----- ERRORS -----")
print(f"Total error cases: {len(errors_small)}")
display(errors_small.head())
print(f"Total results: {len(results_small)}")
display(results_small.head())

In [ ]:
results_small["best_epochs"]

#### Checking the errors
Checking the error cases to see if they can be fixed within the code for the future

In [ ]:
if already_computed_small:
    display(results_small['windows'].drop_duplicates())

In [ ]:
if (already_computed_small):
    errors_small['n_epochs'] = pd.to_numeric(errors_small['n_epochs'], errors='coerce').astype('Int64')
    errors_small['n_windows'] = pd.to_numeric(errors_small['n_windows'], errors='coerce').astype('Int64')
    errors_small['maskared_percent'] = pd.to_numeric(errors_small['maskared_percent'], errors='coerce').astype(float)
    errors_small['error'] = errors_small['error'].astype(str)
    print(results_small.dtypes)
    results_small['model_size'] = results_small['model_size'].astype(pd.StringDtype())
    print("--- Second check ---")
    print(results_small.dtypes)

In [ ]:
if already_computed_small:
    display(results_small['windows'].drop_duplicates())

In [ ]:
if errors_small.shape[0] > 0:
    error_small_window_sizes = list(errors_small['window'].drop_duplicates())
    display(error_small_window_sizes)

In [ ]:
if errors_small.shape[0] > 0:
    error_small_mssg =errors_small['error'].astype(str).drop_duplicates()
    display(error_small_mssg)

We see two failures to check within the failures:
1) Windows do no respect the requested distance between sepparated windows (only one with ne next). TODO: check
2) This dataset needs windows bigger than 4 for MOMENT - Small. => 
    => We need:

   - A minimum and maximum variate allowing to ask for windows sizes inside an interval
   - Control within the windows sizes. If we all like this log table, we can save an unique variate (not saving the windows part) just to check if a window has already failed with this error so it does not execute again.
   - ¿Buen TFG un SQL de gestión de errores para DeepVATS? -> Hablar con Víctor

First valid window:

In [ ]:
small_windows = results_small['windows'].drop_duplicates()
print(small_windows.shape)
display(small_windows)

In [ ]:
filtered_windows = small_windows.apply(lambda x: greater_than(x, 5)).apply(sorted)
filtered_windows

A futuros, se observa que, cuando analicemos este dataset, deberemos:
- Usar ventanas mayores que 5, preferiblemente, >= 8.
- Corregir la función de ventanas para que indique en un warning y en una variable el número de ventanas devuelto realmente. Gestionar para que si no se ha devuelto el número de ventanas esperado, se corte el loop ahí en lugar de repetir los mismos expserimentos. 

#### Time analysis

In [ ]:
total_time = results_small['time'].sum()
print(f"{total_time} seconds")
print(f"{total_time/60} minutes")
print(f"{total_time/60/60} hours")

In [ ]:
results_small.plot(figsize = (10,4))

In [ ]:
df_time_analysis_small = results_small[["time", "n_epochs", "dataset_percent", "maskared_percent", "n_windows", "last_train_loss", 'last_mse']]
df_time_analysis_small.plot(figsize = (10,4))

In [ ]:
plot_with_multiple_secondary_y(df_time_analysis_small, ["n_epochs", "time", "n_windows"], ["last_train_loss", "dataset_percent", "maskared_percent", 'last_mse'])

In [ ]:
profile_small_df = df_time_analysis_small.copy(deep = True)

In [ ]:
#profile_small.to_notebook_iframe()

In [ ]:
profile_small_df.head()

In [ ]:
print(profile_small_df.dtypes)

In [ ]:
# Convertir n_epochs y n_windows a enteros, manejando posibles errores
profile_small_df['n_epochs'] = pd.to_numeric(profile_small_df['n_epochs'], errors='coerce').astype('Int64')
profile_small_df['n_windows'] = pd.to_numeric(profile_small_df['n_windows'], errors='coerce').astype('Int64')

# Convertir maskared_percent a float, manejando posibles errores
profile_small_df['maskared_percent'] = pd.to_numeric(profile_small_df['maskared_percent'], errors='coerce').astype(float)

In [ ]:
profile_small = ydp.ProfileReport(profile_small_df, title="Pandas Profiling Report for 'df_time_analysis_small'", explorative=True)

In [ ]:
 plot_correlation(profile_small_df)

A nivel de tiempo, se observa que:

- Apenas afecta el enmascarado, con una correlación negativa con el last loss. Lo mismo ocurre con el n_epochs
- Más tiempo parece hacer crecer el last_loss
- Lo que más influye es el porcentaje de dataset utilizado para el fine_tuning. Más dataset parece hacer empeorar por alguna razón
- El last loss está bastante relacionado con el tiempo dedicado al fine-tuning, como es de esperar. Pero. La correlación es positiva => más last loss => más tiempo.
- El número de ventanas parece ir en contra de bajar el loss.. quizá porque va de la mano del número de ventanas.


### Loss & metrics analysis

A nivel de losses, se observa (sin tener en cuenta mse, rmse, mae, smape): 
- Muy poca relación con el enmascarado, cosa que de primeras sorprende
- Mucha relación con el % de dataset utilizado en el fine-tuning
Por lo tanto,
    - vamos a filtrar el dataset para tener tiempos menores a 8 segundos
y buenos losses.
    - Veamos a partir de qué momento se obtienen unos losses razonables  en %.

In [ ]:
results_small.columns

In [ ]:
df_loss_small = results_small[["time", "n_epochs", "dataset_percent", "maskared_percent", "n_windows", "first_train_loss", "last_train_loss", "first_mse", "last_mse"]].copy()

In [ ]:
df_loss_small["loss_train_percent"] = (df_loss_small['first_train_loss']-df_loss_small['last_train_loss'])*100/(df_loss_small['first_train_loss'])
df_loss_small["loss_train_percent"].plot()
df_loss_small["mse_percent"] = (df_loss_small['first_mse']-df_loss_small['last_mse'])*100/(df_loss_small['first_mse'])
df_loss_small["mse_percent"].plot()

In [ ]:
df_loss_small[df_loss_small["time"] < 8]["loss_train_percent"].plot()
df_loss_small[df_loss_small["time"] < 8]["mse_percent"].plot()

In [ ]:
# Filtrar filas donde "time" es menor a 8 segundos
df_loss_small_best_cases = df_loss_small[df_loss_small["time"] < 8].copy()

print("--------------------- Train -----------------")
# Filtrar los 5 mejores loss_percents
display(df_loss_small_best_cases.sort_values('loss_train_percent', ascending = False)[:5])
# Filtrar los 5 mejores loss_percent con dataset percent <= 0.25
display(df_loss_small_best_cases[df_loss_small_best_cases['dataset_percent'] < 0.50].sort_values('loss_train_percent', ascending = False)[:5])
print("---------------------- Eval ------------------")
# Filtrar los 5 mejores loss_percents
display(df_loss_small_best_cases.sort_values('mse_percent', ascending = False)[:5])
# Filtrar los 5 mejores loss_percent con dataset percent <= 0.25
display(df_loss_small_best_cases[df_loss_small_best_cases['dataset_percent'] < 0.50].sort_values('loss_train_percent', ascending = False)[:5])

In [ ]:
# Convertir n_epochs y n_windows a enteros, manejando posibles errores
profile_small_df = df_loss_small[['time', 'n_epochs', 'dataset_percent', 'maskared_percent', 'n_windows', 'loss_train_percent', 'mse_percent']].copy()
profile_small_df.dtypes

In [ ]:
# Convertir n_epochs y n_windows a enteros, manejando posibles errores
profile_small_df['n_epochs'] = pd.to_numeric(profile_small_df['n_epochs'], errors='coerce').astype('Int64')
profile_small_df['n_windows'] = pd.to_numeric(profile_small_df['n_windows'], errors='coerce').astype('Int64')
# Convertir maskared_percent a float, manejando posibles errores
profile_small_df['maskared_percent'] = pd.to_numeric(profile_small_df['maskared_percent'], errors='coerce').astype(float)

In [ ]:
profile_small = ydp.ProfileReport(profile_small_df, title="Pandas Profiling Report for 'df_loss_small'", explorative=True)
plot_correlation(profile_small_df)

#### What if I focus on 0.25 dataset percent?

In [ ]:
profile_small_df_2 = df_time_analysis_small[df_time_analysis_small['dataset_percent'] == 0.25].drop(columns=['dataset_percent', 'time']).copy()

In [ ]:
profile_small_df_2.dtypes

In [ ]:
# Convertir n_epochs y n_windows a enteros, manejando posibles errores
profile_small_df_2['n_epochs'] = pd.to_numeric(profile_small_df_2['n_epochs'], errors='coerce').astype('Int64')
profile_small_df_2['n_windows'] = pd.to_numeric(profile_small_df_2['n_windows'], errors='coerce').astype('Int64')
# Convertir maskared_percent a float, manejando posibles errores
profile_small_df_2['maskared_percent'] = pd.to_numeric(profile_small_df_2['maskared_percent'], errors='coerce').astype(float)

In [ ]:
profile_small_2 = ydp.ProfileReport(profile_small_df_2, title="Pandas Profiling Report for 'df_time_analysis_small' for < 0.25 dataset percent", explorative=True)
plot_correlation(profile_small_df_2)

* Correlación inversa entre maskared percent y last loss * => Aumentar maskared percent baja last_loss
* => Es parte de lo que queremos!

In [ ]:
#! ls errors*

In [ ]:
print(torch.cuda.memory_allocated())
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())

In [ ]:
before = torch.cuda.memory_allocated()
print(before)
moment_small.to('cpu')
for param in moment_small.parameters():
    param.to('cpu')
after = torch.cuda.memory_allocated()
print(after)
print("Mejorado: ", after-before)

## Moment-base

### Download the model

In [ ]:
print("Getting base artifact: ", enc_artifact_base_name)
enc_artifact_base  = wandb_api.artifact(enc_artifact_base_name, type='learner')
moment_base  = enc_artifact_base.to_obj()

### Select parameters

In [ ]:
total_cases_small      = len(n_epochs_list)*len(dataset_percents)*len(maskared_percents)*len(sizes)
print(f"Total cases: {total_cases_small}")
n_epochs_list_base= [5, 10, 20]
dataset_percents_base  = [0.25, 0.5]
maskared_percents_base = [ 0.25, 0.5, 0.75]
sizes_base             = [1, 5, 10]
total_cases_base = len(n_epochs_list_base)*len(dataset_percents_base)*len(maskared_percents_base)*len(sizes)
print(f"Total cases: {total_cases_base}")
expected_time = total_time*total_cases_base/total_cases_small
print(f"Expected time: {expected_time} seconds | {expected_time/60} minutes | {expected_time/60/60} hours")

### Configure files and wether already computed or not

In [ ]:
already_computed_base = True
results_base = None
errors_base = None
file_errors_base = 'errors_base_24012025_1.csv'
file_results_base = 'results_base_24012025_1.csv'

### Execute cases

In [ ]:
results_base = None
errors_base = None
if already_computed_base:
    results_base = pd.read_csv(file_results_base, index_col=None, header=0)
    errors_base = pd.read_csv(file_errors_base, index_col=None, header=0)
else:
    results_base, errors_base = cases_loop(
        model             = moment_base, 
        n_epochs_list     = n_epochs_list_base, 
        dataset_percents  = dataset_percents_base, 
        maskared_percents = maskared_percents_base,
        n_sizes_list      = sizes_base, 
        summarized        = True,
        verbose           = 8
    )
    results_base.to_csv(file_results_base, index=False, header=True)
    errors_base.to_csv(file_errors_base, index=False, header=True)
    already_computed_base = True

In [ ]:
print("----- ERRORS -----")
print(f"Total error cases: {len(errors_base)}")
display(errors_base.head())
print(f"Total results: {len(results_base)}")
display(results_base.head())

### Checking the errors

In [ ]:
len(results_base)

In [ ]:
error_base_window_sizes = list(errors_base['windows'].drop_duplicates())
error_base_window_sizes

In [ ]:
error_base_mssg = errors_base['error'].astype(str).drop_duplicates()
error_base_mssg

In [ ]:
base_windows = results_base['windows'].drop_duplicates()
filtered_windows_base = base_windows.apply(lambda x: greater_than(x, 5)).apply(sorted)
filtered_windows_base

#### Time analysis

In [ ]:
df_time_analysis_base = results_base[["time", "n_epochs", "dataset_percent", "maskared_percent", "n_windows", "last_train_loss", "last_mse"]]

In [ ]:
plot_with_multiple_secondary_y(df_time_analysis_base, ["n_epochs", "time", "n_windows"], ["last_train_loss", "dataset_percent", "maskared_percent", "last_mse"])

In [ ]:
profile_base_df = df_time_analysis_base.copy(deep = True)
profile_base = ydp.ProfileReport(profile_base_df, title="Pandas Profiling Report for 'df_time_analysis_base'", explorative=True)

In [ ]:
plot_correlation(profile_base_df)

#### Loss & metrics analysis

In [ ]:
df_loss_base = results_base[["time", "n_epochs", "dataset_percent", "maskared_percent", "n_windows", "first_train_loss", "last_train_loss", "first_mse", "last_mse"]].copy()
df_loss_base["loss_percent"] = (df_loss_base['first_train_loss']-df_loss_base['first_train_loss'])*100/(df_loss_base['first_train_loss'])
df_loss_base["loss_percent"].plot()
df_loss_base["mse_percent"] = (df_loss_base['first_mse']-df_loss_base['last_mse'])*100/(df_loss_base['first_mse'])
df_loss_base["mse_percent"].plot()

In [ ]:
df_loss_base[df_loss_base["time"] < 8]["loss_percent"].plot()
df_loss_base[df_loss_base["time"] < 8]["mse_percent"].plot()

In [ ]:
df_loss_base[df_loss_base["time"] < 20]

In [ ]:
# Filtrar filas donde "time" es menor a 8 segundos
df_loss_base_best_cases = df_loss_base[df_loss_base["time"] < 8].copy()

print("---- Mejoras en el entrenamiento ----")
# Filtrar los 5 mejores loss_percents
display(df_loss_base_best_cases.sort_values('loss_percent', ascending = False)[:5])
# Filtrar los 5 mejores loss_percent con dataset percent <= 0.25
display(df_loss_base_best_cases[df_loss_base_best_cases['dataset_percent'] < 0.50].sort_values('loss_percent', ascending = False)[:5])
print("---- Mejoras en la validación  ----")
# Filtrar los 5 mejores loss_percents
display(df_loss_base_best_cases.sort_values('mse_percent', ascending = False)[:5])
# Filtrar los 5 mejores loss_percent con dataset percent <= 0.25
display(df_loss_base_best_cases[df_loss_base_best_cases['dataset_percent'] < 0.50].sort_values('mse_percent', ascending = False)[:5])

In [ ]:
print(torch.cuda.memory_allocated())
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())

## Moment-large

### Download de large model

In [ ]:
already_computed_large = True
file_errors_large = 'errors_large_24012025_1.csv'
file_results_large = 'results_large_24012025_1.csv'

In [ ]:
if not already_computed_large:
    print("Getting large artifact: ", enc_artifact_large_name)
    enc_artifact_large = wandb_api.artifact(enc_artifact_large_name, type='learner')
    print(enc_artifact_large.name)
    moment_large = enc_artifact_large.to_obj()
    print(count_parameters(moment_large))

In [ ]:
total_cases_small      = len(n_epochs_list)*len(dataset_percents)*len(maskared_percents)*len(sizes)
print(f"Total cases: {total_cases_small}")
total_cases_base = len(n_epochs_list_base)*len(dataset_percents_base)*len(maskared_percents_base)*len(sizes)
print(f"Total cases: {total_cases_base}")
expected_time = total_time*total_cases_base/total_cases_small
print(f"Expected time: {expected_time} seconds | {expected_time/60} minutes | {expected_time/60/60} hours")

n_epochs_list_large     = [5, 10, 20]
dataset_percents_large  = [0.25, 0.5] # No tendría sentido porque sería como hacer lo mismo que con mvp
maskared_percents_large = [0.25, 0.5, 0.75]
sizes_large             = [1, 5, 10]
print(f"Total cases: {len(n_epochs_list)*len(dataset_percents)*len(maskared_percents)*len(sizes)}")

In [ ]:
torch.cuda.memory_allocated()
torch.cuda.set_device(1)
torch.cuda.current_device()

In [ ]:
results_large = None
errors_large = None
if already_computed_large:
    results_large = pd.read_csv(file_results_large, index_col=None, header=0)
    errors_large = pd.read_csv(file_errors_large, index_col=None, header=0)
else:
    results_large, errors_large = cases_loop(
        model             = moment_large, 
        n_epochs_list     = n_epochs_list_large,
        dataset_percents  = dataset_percents_large, 
        maskared_percents = maskared_percents_large, 
        n_sizes_list      = sizes_large, 
        summarized        = True,
        save              = True,
        file_errors       =  file_errors_large,
        file_results      = file_results_large
    )
    already_computed_large = True

In [ ]:
print("----- ERRORS -----")
print(f"Total error cases: {len(errors_large)}")
display(errors_large.head())
print(f"Total results: {len(results_large)}")
display(results_large.head())
print("------------------")

In [ ]:
print("----- ERRORS -----")
print(f"Total error cases: {len(errors_large)}")
display(errors_large.head())
print(f"Total results: {len(results_large)}")
display(results_large.head())

In [ ]:
before = torch.cuda.memory_allocated()
print(before)
moment_large.to('cpu')
for param in moment_large.parameters():
    param.to('cpu')
after = torch.cuda.memory_allocated()
print(after)
gc.collect()
print("Mejorado: ", after-before)

#### Errors analysis

In [ ]:
if len(errors_large) > 0:
    error_large_window_sizes = list(errors_large['window'].drop_duplicates())
    error_large_window_sizes

In [ ]:
try:
    error_large_mssg = errors_large['error'].astype(str).drop_duplcates()
    display(error_large_mssg)
except Exception as e:
    print(e)

In [ ]:
large_windows = results_large['windows'].drop_duplicates()
print(large_windows.shape)
display(large_windows)

In [ ]:
filtered_windows_large = large_windows.apply(lambda x: greater_than(x, 5)).apply(sorted)
display(filtered_windows_large)

#### Time analysis

In [ ]:
df_time_analysis_large = results_large[["time", "n_epochs", "dataset_percent", "maskared_percent", "n_windows", "last_train_loss", 'last_mse']]
df_time_analysis_large.plot(figsize = (10,4))

In [ ]:
plot_with_multiple_secondary_y(df_time_analysis_large, ["n_epochs", "time", "n_windows"], ["last_train_loss", 'last_mse', "dataset_percent", "maskared_percent"])

In [ ]:
profile_large_df = df_time_analysis_large.copy(deep = True)
profile_large = ydp.ProfileReport(profile_large_df, title = "Pandas Profiling Report for 'df_time_analysis_large'", explorative = True)
#profile_large.to_notebook_iframe()

In [ ]:
plot_correlation(profile_large_df)

In [ ]:
df_loss_large = results_large[["time", "n_epochs", "dataset_percent", "maskared_percent", "n_windows", "first_train_loss", "last_train_loss", "first_mse", "last_mse"]].copy()

In [ ]:
df_loss_large["loss_train_percent"] = (df_loss_large['first_train_loss']-df_loss_large['last_train_loss'])*100/(df_loss_large['first_train_loss'])
df_loss_large["mse_percent"] = (df_loss_large['first_mse']-df_loss_large['last_mse'])*100/(df_loss_large['first_mse'])
df_loss_large["loss_train_percent"].plot()
df_loss_large["mse_percent"].plot()

In [ ]:
df_loss_large_best_cases = df_loss_large[df_loss_large["time"] < 8].copy()
display(df_loss_large_best_cases)
print("--- train ---")
display(df_loss_large_best_cases[df_loss_large_best_cases["loss_train_percent"] > 0])
print("--- eval ---")
display(df_loss_large_best_cases[df_loss_large_best_cases["mse_percent"] > 0])
display(df_loss_large_best_cases)

In [ ]:
display(df_loss_large_best_cases.sort_values('loss_train_percent', ascending = False))

In [ ]:
before = torch.cuda.memory_allocated()
print(before)
gc.collect()
torch.cuda.empty_cache()
after = torch.cuda.memory_allocated()
print(after)
print(after-before)

In [ ]:
gpu_tensors = []
for obj in gc.get_objects():
    try:
        if isinstance(obj, torch.Tensor) and obj.is_cuda:
            gpu_tensors.append(obj)
    except ReferenceError:
        continue # Omitir los objetos que ya han sido recolectados
print(len(gpu_tensors))

#### Loss & metrics analysis

In [ ]:
#| export
if verbose > 0: print("Execution ended")
beep(1)
beep(1)
beep(1)
beep(1)
beep(1)

In [ ]:
#| hide
if reset_kernel:
    os._exit(00)

# Memory checks

In [ ]:
snapshot = torch.cuda.memory_snapshot()
gpu_tensors = [obj['tensor'] for obj in snapshot if 'tensor' in obj]
for tensor_info in gpu_tensors:
    print(f"Size: {tensor_info['size']}, Device: {tensor_info['device']}, Data type: {tensor_info['dtype']}")
print(gpu_tensors)

In [ ]:
object_types = set()
for obj in snapshot:
    for key in obj.keys():
        object_types.add(key)
print("Tipos de objetos encontrados en el snapshot:")
for obj_type in sorted(object_types):
    print(obj_type)
heavier_obj = None
for obj in snapshot:
    if heavier_obj is None or obj['total_size'] > heavier_obj['total_size']: heavier_obj = obj
print(heavier_obj)

In [ ]:
before = torch.cuda.memory_allocated()
print(torch.cuda.memory_allocated())
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())
after = torch.cuda.memory_allocated()
print(after)
print("Mejorado: ", after-before)

In [ ]:
before = torch.cuda.memory_allocated()
print(before)
moment_small.to('cpu')
for param in moment_small.parameters():
    param.to('cpu')
after = torch.cuda.memory_allocated()
print(after)
print("Mejorado: ", after-before)

In [ ]:
before = torch.cuda.memory_allocated()
print(before)
moment_base.to('cpu')
for param in moment_base.parameters():
    param.to('cpu')
after = torch.cuda.memory_allocated()
print(after)
print("Mejorado: ", after-before)

In [ ]:
before = torch.cuda.memory_allocated()
print(before)
moment_large.to('cpu')
for param in moment_large.parameters():
    param.to('cpu')
after = torch.cuda.memory_allocated()
print(after)
print("Mejorado: ", after-before)

In [ ]:
error_small_window_sizes = list(errors_small_hoy['windows'].drop_duplicates())
error_small_window_sizes

In [ ]:
errors_small_hoy

In [ ]:
total_time = results_small_hoy['time'].sum()
print(f"{total_time} seconds")
print(f"{total_time/60} minutes")
print(f"{total_time/60/60} hours")

In [ ]:
df_time_analysis_small = results_small_hoy[["time", "n_epochs", "dataset_percent", "maskared_percent", "last_train_loss", 'last_mse']]
df_time_analysis_small.plot(figsize = (10,4))

In [ ]:
plot_with_multiple_secondary_y(df_time_analysis_small, ["n_epochs", "time"], ["last_train_loss", "dataset_percent", "maskared_percent", 'last_mse'])

In [ ]:
profile_small_df = df_time_analysis_small.copy(deep = True)

In [ ]:
print("--- Cases ---")
print("--- Number of epochs ---")
print(profile_small_df["n_epochs"].unique())
print("--- Dataset percent ---")
print(profile_small_df["dataset_percent"].unique())
print("--- Dataset maskared percent ---")
print(profile_small_df["maskared_percent"].unique())

In [ ]:
profile_small_df.head()

In [ ]:
print(profile_small_df.dtypes)

In [ ]:
# Convertir n_epochs y n_windows a enteros, manejando posibles errores
profile_small_df['n_epochs'] = pd.to_numeric(profile_small_df['n_epochs'], errors='coerce').astype('Int64')
#profile_small_df['n_windows'] = pd.to_numeric(profile_small_df['n_windows'], errors='coerce').astype('Int64')

# Convertir maskared_percent a float, manejando posibles errores
profile_small_df['maskared_percent'] = pd.to_numeric(profile_small_df['maskared_percent'], errors='coerce').astype(float)

In [ ]:
profile_small = ydp.ProfileReport(profile_small_df, title="Pandas Profiling Report for 'df_time_analysis_small'", explorative=True)

In [ ]:
 plot_correlation(profile_small_df)